# Notebook Test Cases 
It is a notebook which show examples with test cases:
- how you can aggregate test case events and build a table like 'Test Script -> Test Case -> Number of events'
- Latency

In [1]:
import pandas as pd
from IPython.core.display import display, HTML
from pprint import pprint
from datetime import datetime, timedelta
from th2_data_services.data_source import DataSource
from th2_data_services.data import Data
from th2_data_services.events_tree import EventsTree
import th2_data_services_utils.utils as Utils
from pandas import DataFrame, Grouper

# This settings for increase display jupyter notebook and dataframe table.
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.options.display.max_rows = 550
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', 1000)

In [2]:
# For understand which event type on based name we get from stream.
def get_super_type(record, tree):
    name = record.get("eventName")
    parent_id = record.get("parentEventId")
    super_type = record.get("eventType")
    if super_type == "":
        if "Recon" in name:
            super_type = "Recon Folder"
        else:
            if not parent_id:
                super_type = "Test Run"
            else:
                parent_event = tree.get(parent_id)
                if parent_event:
                    parent_super_type = get_super_type(parent_event, tree)
                    if parent_super_type == "Test Run":
                        super_type = "Test Case"
                    elif parent_super_type == "Recon Folder":
                        super_type = "Recon Rule"
                    elif parent_super_type == "Recon Rule":
                        super_type = "Recon Status"
                    elif parent_super_type == "Recon Status":
                        super_type = "Recon Event"

    return super_type

# Base extract (transform function)
# record is required arguments.
def extract_basic(record):
    new_object = {}
    start_time = datetime.fromtimestamp(record.get("startTimestamp", {}).get("epochSecond", 0))
    start_time += timedelta(microseconds=record.get("startTimestamp", {}).get("nano", 0)/1000)
    end_time = datetime.fromtimestamp(record.get("endTimestamp", {}).get("epochSecond", 0))
    end_time += timedelta(microseconds=record.get("endTimestamp", {}).get("nano", 0)/1000)
    new_object.update(
        {
            "start_time": start_time,
            "end_time": end_time,
            "super_type": get_super_type(record, tree),
            "eventName": record.get("eventName"),
            "parentEventId": record.get("parentEventId"),
            "status": "SUCCESSFUL" if record.get("successful") else "FAILED",
            "body": record.get("body"),
            "attachedMessageIds": record.get("attachedMessageIds")
        }
    )
    return new_object

## Create Data Source object
The DataSource object lets you retrieve data in the easiest way.

NOTE: You can change the URL via eponymous property of this object.

In [3]:
START_TIME = datetime(year=2021, month=6, day=20, hour=10, minute=44, second=41, microsecond=692724)
END_TIME = datetime(year=2021, month=6, day=20, hour=10, minute=45, second=49, microsecond=28579)

DEMO_HOST = "10.64.66.66"  # th2-kube-demo  Host port where rpt-data-provider is located.
DEMO_PORT = "30999"  # Node port of rpt-data-provider.
data_source = DataSource(F"http://{DEMO_HOST}:{DEMO_PORT}")

events_with_attached_msgs = data_source.get_events_from_data_provider(
    startTimestamp=START_TIME,
    endTimestamp=END_TIME,
    metadataOnly=False,
    attachedMessages=True,
)

## This example demonstrates events retrieving


In [4]:
# We build events tree for further assistance.
events_tree = EventsTree(events_with_attached_msgs)
tree = events_tree.events

# Here we get events which doesn't exist in data source interval.
events_tree.recover_unknown_events(data_source)

## [1] Basic statistics by test cases
Shows how many test case are failed and are passed.

In [5]:
def transform_output(record):    
    new_obj = {
        "Test Case": 1,
        "Status": record.get("status")
    }
    return new_obj

events = data_source.get_events_from_data_provider(
    startTimestamp=START_TIME,
    endTimestamp=END_TIME,
    metadataOnly=False
)
data = events\
        .map(extract_basic)\
        .filter(lambda record: record.get("super_type") == "Test Case")\
        .map(transform_output)

# From pandas for comforted view
df = DataFrame(data=data)
# display_html(df)
df = df.groupby(["Status"]).sum()
df["Percent"] = df["Test Case"] / df["Test Case"].sum() * 100
df

,Test Case,Percent
Status,,
FAILED,3,50.0
SUCCESSFUL,3,50.0


## [2] Detailed statistics by test cases
Shows each test case name, status, time and duration.

In [8]:
def ancestor_is_test_case(record):
    if not record.get("parentEventId"):
        return False
    ancestor = events_tree.get_ancestor_by_super_type(record, "Test Case", get_super_type)
    if ancestor:
        return True
    return False

def transform_output(record):
    test_case = events_tree.get_ancestor_by_super_type(record, "Test Case", get_super_type)
    test_run = tree.get(test_case.get("parentEventId"))
    
    start_time = datetime.fromtimestamp(test_case.get("startTimestamp", {}).get("epochSecond", 0))
    start_time += timedelta(microseconds=test_case.get("startTimestamp", {}).get("nano", 0)/1000)
    
    message_id = record.get("attachedMessageIds")
    
    if not message_id:
        return None
    
    message = data_source.find_messages_by_id_from_data_provider(message_id)
    if not message:
        return None
    
    body = message[0].get("body", {})
    if not body:
        return None
    
    end_time = body.get("metadata", {}).get("timestamp")
    end_time = datetime.strptime(end_time, "%Y-%m-%dT%H:%M:%S.%fZ")
    end_time += timedelta(hours=3)
    
    new_obj = {
        "Test Run": test_run.get("eventName"),
        "Test Case": test_case.get("eventName"),
        "Status": "SUCCESSFUL" if test_case.get("successful") else "FAILED",
        'Start Time': start_time,
        'End Time': end_time,
    }
    return new_obj

data = events_with_attached_msgs\
        .map(extract_basic)\
        .filter(ancestor_is_test_case)\
        .filter(lambda record: record.get("super_type") in ["Verification", "message"])\
        .map(transform_output)

df = DataFrame(data=data)
df = df.groupby(["Test Run", "Test Case", "Status"]).agg({"Start Time": "min", "End Time": "max"}).reset_index()
df["duration"] = df["End Time"] - df["Start Time"]
df.sort_values(by=["Start Time"])

,Test Run,Test Case,Status,Start Time,End Time,duration
0,[TS_1]Aggressive IOC vs two orders: second order's price is lower than first,Case[TC_1.1]: Trader DEMO-CONN1 vs trader DEMO-CONN2 for instrument INSTR1,SUCCESSFUL,2021-06-20 13:44:48.169672,2021-06-20 13:44:58.620,0 days 00:00:10.450328
1,[TS_1]Aggressive IOC vs two orders: second order's price is lower than first,Case[TC_1.2]: Trader DEMO-CONN1 vs trader DEMO-CONN2 for instrument INSTR2,SUCCESSFUL,2021-06-20 13:44:59.848920,2021-06-20 13:45:07.537,0 days 00:00:07.688080
2,[TS_1]Aggressive IOC vs two orders: second order's price is lower than first,Case[TC_1.3]: Trader DEMO-CONN1 vs trader DEMO-CONN2 for instrument INSTR3,SUCCESSFUL,2021-06-20 13:45:07.895991,2021-06-20 13:45:15.547,0 days 00:00:07.651009
3,[TS_1]Aggressive IOC vs two orders: second order's price is lower than first,Case[TC_1.4]: Trader DEMO-CONN1 vs trader DEMO-CONN2 for instrument INSTR4,FAILED,2021-06-20 13:45:16.172487,2021-06-20 13:45:24.647,0 days 00:00:08.474513
4,[TS_1]Aggressive IOC vs two orders: second order's price is lower than first,Case[TC_1.5]: Trader DEMO-CONN1 vs trader DEMO-CONN2 for instrument INSTR5,FAILED,2021-06-20 13:45:24.885771,2021-06-20 13:45:32.529,0 days 00:00:07.643229
5,[TS_1]Aggressive IOC vs two orders: second order's price is lower than first,Case[TC_1.6]: Trader DEMO-CONN1 vs trader DEMO-CONN2 for instrument INSTR6,FAILED,2021-06-20 13:45:32.778780,2021-06-20 13:45:38.298,0 days 00:00:05.519220


## [3] Latency density between messages
 
Searches pairs messages with type NewOrderSingle and ExecutionReport. Then calculates latency and demonstrates on plot.

In [ ]:
def is_new_single_order_or_execution_report(record):
    body = record.get("body")
    if body:
        message_type = body.get("metadata", {}).get("messageType")
        if message_type in ["NewOrderSingle", "ExecutionReport"]:
            return True
    return False

def clear_unnecessery_fields(record):
    new_obj = None
    body = record.get("body")
    if body:
        fields = body.get("fields", {})
        clOrdID = fields.get("ClOrdID", {}).get("simpleValue")
        ord_status = fields.get("OrdStatus", {}).get("simpleValue")
        
        metadata = body.get("metadata", {})
        message_type = metadata.get("messageType")
        session_alias = metadata.get("id", {}).get("connectionId", {}).get("sessionAlias")
        time = metadata.get("timestamp")
        
        new_obj = {
            "clOrdID": clOrdID,
            "OrdStatus": ord_status,
            "MessageType": message_type,
            "sessionAlias": session_alias,
            "time": time,
        }
    return new_obj

streams = set()
for record in events_tree.events.values():
    messages = record.get("attachedMessageIds")
    for msg in messages:
        streams.add(msg.split(":")[0])
        
messages = data_source.get_messages_from_data_provider(
    startTimestamp=START_TIME,
    endTimestamp=END_TIME,
    stream=list(streams)
)
data = messages\
        .filter(is_new_single_order_or_execution_report)\
        .map(clear_unnecessery_fields)

roundtrips = {}
latency = []

for record in data:
    msg_type = record.get("MessageType")
    clOrdID = record.get("clOrdID")
    
    if msg_type == "NewOrderSingle":
        if clOrdID not in roundtrips:
            roundtrips[clOrdID] = record.get("time")
    elif msg_type == "ExecutionReport":
        if record.get("OrdStatus") == '0':
            if clOrdID in roundtrips:
                current_latency = datetime.strptime(record.get("time"), "%Y-%m-%dT%H:%M:%S.%fZ") -  datetime.strptime(roundtrips[clOrdID], "%Y-%m-%dT%H:%M:%S.%fZ")
                latency.append({"latency": 1, "time": datetime.strptime(str(current_latency), "%H:%M:%S.%f")})

df = DataFrame(data=latency).set_index("time").groupby(Grouper(freq="10ms")).sum()
df.index = df.index.strftime("%S.%f")

Utils.create_tick_diagram(df)  # The plot may not be shown if you have not restarted the notebook.

![alt text](latency.png "123")

## [4] Outgoing messages
Calculates quantity of outgoing messages in each test case.

In [9]:
def transform_output(record):
    test_run = events_tree.get_ancestor_by_super_type(record, "Test Run", get_super_type).get("eventName")
    test_case = events_tree.get_ancestor_by_super_type(record, "Test Case", get_super_type).get("eventName")
    new_obj = {
        "Test Run": test_run,
        "Test Case": test_case,
        "Outgoing message": record.get("eventName"),
    }
    return new_obj

data = events_with_attached_msgs\
        .map(extract_basic)\
        .filter(lambda record: record.get("super_type") == "Outgoing message")\
        .map(transform_output)

# Functions from pandas
DataFrame(data=data).groupby(["Test Run", "Test Case", "Outgoing message"]).size().reset_index(name="Numbers")

,Test Run,Test Case,Outgoing message,Numbers
0,[TS_1]Aggressive IOC vs two orders: second order's price is lower than first,Case[TC_1.1]: Trader DEMO-CONN1 vs trader DEMO-CONN2 for instrument INSTR1,Send 'NewOrderSingle' message to connectivity,3
1,[TS_1]Aggressive IOC vs two orders: second order's price is lower than first,Case[TC_1.2]: Trader DEMO-CONN1 vs trader DEMO-CONN2 for instrument INSTR2,Send 'NewOrderSingle' message to connectivity,3
2,[TS_1]Aggressive IOC vs two orders: second order's price is lower than first,Case[TC_1.3]: Trader DEMO-CONN1 vs trader DEMO-CONN2 for instrument INSTR3,Send 'NewOrderSingle' message to connectivity,3
3,[TS_1]Aggressive IOC vs two orders: second order's price is lower than first,Case[TC_1.4]: Trader DEMO-CONN1 vs trader DEMO-CONN2 for instrument INSTR4,Send 'NewOrderSingle' message to connectivity,3
4,[TS_1]Aggressive IOC vs two orders: second order's price is lower than first,Case[TC_1.5]: Trader DEMO-CONN1 vs trader DEMO-CONN2 for instrument INSTR5,Send 'NewOrderSingle' message to connectivity,3
5,[TS_1]Aggressive IOC vs two orders: second order's price is lower than first,Case[TC_1.6]: Trader DEMO-CONN1 vs trader DEMO-CONN2 for instrument INSTR6,Send 'NewOrderSingle' message to connectivity,1
